# Task 4.1 Demonstrate execution Modes

## Objective 1: Execute on hardware

### Overview

Executing quantum circuits on real hardware involves several key steps:
1. **Circuit Preparation**: Create and optimize quantum circuits initialized locally or on cloud
2. **Backend Selection**: Choose appropriate quantum hardware
3. **Job Submission**: Send circuits to the quantum processor in batch or session
4. **Result Retrieval**: Obtain and analyze measurement outcomes

## Objective 2: Execution modes

### Three Execution Modes

IBM Quantum provides three distinct execution modes for running quantum programs:

1. **Job Mode** (`mode=backend`)
   - Single request of Sampler or Estimator without a context manager
   - Simplest mode for one-off executions
   - Specify `mode=backend` when initializing the primitive 

2. **Batch Mode** (`mode=batch`)
   - Multi-job workload made up of independently executable jobs, no condoitional relations to each other
   - Jobs are scheduled together for efficiency
   - Not gauranteed to run in order of submission, may run in parallel to other users jobs , QPU calibriation jobs may also run between batched jobs
   - Queuing time doesn't decrease for the first job submitted in a job
   - Specify `mode=batch` when initializing the primitive

3. **Session Mode** (`mode=session`)
   - Dedicated window to run a multi-job workload
   - execlusive access to the QPU and other jobs can run including calibriation jobs
   - Particularly useful for iterative tasks that require frequent communication between classical and quantum resources
   - Queuing time doesn't decrease for the first job submitted in a job
   - Specify `mode=session` when initializing the primitive

## Objective 3: Choose execution mode

### When to Use Each Mode

| Mode | Best For | Key Features |
|------|----------|-------------|
| **Batch** | Large collections of independent circuits | • Entire batch scheduled together<br>• Classical computations run in parallel<br>• Minimal delay between jobs<br>• Flexible - cancel remaining jobs if needed<br>• Less expensive than sessions |
| **Session** | Variational algorithms, iterative workloads | • All batch mode functionality<br>• Dedicated access to QPU<br>• Useful when inputs arrive gradually<br>• Better for workloads requiring low latency |
| **Job** | Small experiments, quick tests | • Simplest interface<br>• Might run sooner than batch<br>• Good for learning and testing |

### Best Practices

- Use batch mode by default, especially when all inputs are ready, it is more cost-effective than sessions.

- Batch mode is ideal for submitting multiple primitive jobs at once to reduce total runtime.

- Session mode is best for iterative workflows or when dedicated QPU access is required, but it is generally more expensive.

- Job mode should be used only for a single primitive request.

- Open Plan users cannot use sessions.

### Recommendations:

- Each job has fixed overhead. If a job uses less than 1 minute of QPU time, combine multiple tasks into a single job.

- If jobs take more than 1 minute or cannot be combined, submit multiple jobs in batch or session mode to exploit parallel classical processing (up to 5 jobs), even though the QPU runs one job at a time.

- Tune workload size and parallelism specially in sessions to minimize wall-clock time and cost; splitting large jobs can be more efficient.

## Objective 4 : Run Jobs in Batch

### Batch Execution Fundamentals

Batch mode allows you to submit multiple quantum jobs that are scheduled together. This is more efficient than submitting individual jobs separately.

In [1]:
from qiskit_ibm_runtime import (
    QiskitRuntimeService,
    Batch,
    SamplerV2 as Sampler,
    EstimatorV2 as Estimator,
)

# Initialize the runtime service
service = QiskitRuntimeService()

### Open a batch

Can be done by initializing the `Batch` class or by using Context Manager `with Batch()`

In [2]:
# Method 1: Explicit batch creation and closure
backend = service.least_busy(operational=True, simulator=False)

#Create a batch execution context
batch = Batch(backend=backend, max_time="5m")

# Configure primitives to use batch mode
estimator = Estimator(mode=batch)
sampler = Sampler(mode=batch)

# Close the batch explicitly (no context manager used)
batch.close()

In [3]:
# Method 2: Using context manager
from qiskit_ibm_runtime import Batch, SamplerV2 as Sampler, EstimatorV2 as Estimator
 
backend = service.least_busy(operational=True, simulator=False)
# Automatically handle batch lifecycle
with Batch(backend=backend, max_time="5m"):
    estimator = Estimator()  
    sampler = Sampler()

#Batch automatically closes when context exits

#### Batch Duration Limits

max TTL can be defined with `max_time` , it starts when the job batch starts, when max time is reached , the batch is closed and any job running will finish but all queued jobs will fail.

| Plan Type | Maximum Batch Duration |
|-----------|-----------------------|
| Open Plan | 10 minutes |
| Paid Plans | 8 hours |

Additionlly there is also interactive TTL can not be configured (1 minute for all plans), if there is no jobs on the queue within the window, the job is temporarliy deactivated

**Note:** These limits apply to the total execution time of all jobs within a batch.

### Close a batch

A batch automatically closes when its context manager exits. After that the batch stops accepting new jobs and finishes running all queued or active jobs until the maximum TTL is reached, then which it closes permanently.

If you are not using a context manager, you must close the batch manually. Leaving a batch open and submitting jobs later risks hitting the maximum TTL before they start, causing cancellation. Calling `batch.close()` prevents new submissions while allowing already submitted jobs to finish and their results to be retrieved.

In [ ]:
with Batch(backend=backend) as batch:
    estimator = Estimator()
    sampler = Sampler()
    job1 = estimator.run([estimator_pub])
    job2 = sampler.run([sampler_pub])
 
# The batch is no longer accepting jobs but the submitted job will run to completion.
result = job1.result()
result2 = job2.result()

In [ ]:
batch = Batch(backend=backend)
 
# If using qiskit-ibm-runtime earlier than 0.24.0, change `mode=` to `batch=`
estimator = Estimator(mode=batch)
sampler = Sampler(mode=batch)
job1 = estimator.run([estimator_pub])
job2 = sampler.run([sampler_pub])
print(f"Result1: {job1.result()}")
print(f"Result2: {job2.result()}")
 
# Manually close the batch. Running and queued jobs will run to completion.
batch.close()

### Practical Example: Running Circuits in Batch

**Review**
**Note:** Open Plan accounts can not run batches<br/>
The code below will return an error if you use an Open plan account


In [ ]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp

# Create example circuits and observables
qc1 = QuantumCircuit(2)
qc1.h(0)
qc1.cx(0, 1)

qc2 = QuantumCircuit(2)
qc2.h(0)
qc2.cx(0, 1)
qc2.rz(0.5, 1)

# Create observables for estimator
observable1 = SparsePauliOp("ZZ")
observable2 = SparsePauliOp("XX")

# Create primitive units (pubs) for execution
estimator_pub = (qc1, observable1)
sampler_pub = qc2

In [ ]:
# Execute circuits using batch mode with context manager
with Batch(backend=backend) as batch:
    estimator = Estimator()
    sampler = Sampler()
    
    # Submit jobs to batch
    estimator_job = estimator.run([estimator_pub])
    sampler_job = sampler.run([sampler_pub])
 
# Results are available after batch context closes
# The batch is no longer accepting jobs but submitted jobs run to completion
estimator_result = estimator_job.result()
sampler_result = sampler_job.result()

print("Batch execution completed successfully!")
print(f"Estimator result: {estimator_result}")
print(f"Sampler result: {sampler_result}")

### Batch Information and Monitoring

In [ ]:
# Get batch details for monitoring
with Batch(backend=backend) as batch:
    details = batch.details()
    
print("Batch Details:")
print(f"ID: {details['id']}")
print(f"Backend: {details['backend_name']}")
print(f"State: {details['state']}")
print(f"Max Time: {details['max_time']} seconds")
print(f"Accepting Jobs: {details['accepting_jobs']}")
print(f"Mode: {details['mode']}")

### Handling Large Batches of Circuits

In [ ]:
from qiskit.circuit.random import random_circuit

# Create a large number of circuits
max_circuits_per_batch = 10  # Limit per batch submission
total_circuits = 50
circuits = [random_circuit(3, 3) for _ in range(total_circuits)]

# Partition circuits into manageable batches
all_partitioned_circuits = []
for i in range(0, len(circuits), max_circuits_per_batch):
    all_partitioned_circuits.append(circuits[i : i + max_circuits_per_batch])

jobs = []

# Submit partitioned batches
with Batch(backend=backend):
    sampler = Sampler()
    for i, partitioned_circuits in enumerate(all_partitioned_circuits):
        job = sampler.run(partitioned_circuits)
        jobs.append(job)
        print(f"Submitted batch {i+1} with {len(partitioned_circuits)} circuits")

print(f"\nCreated {total_circuits} circuits")
print(f"Partitioned into {len(all_partitioned_circuits)} batches of maximum {max_circuits_per_batch} circuits each")
print(f"Successfully submitted {len(jobs)} jobs to batch")